In [13]:
import requests
from dotenv import load_dotenv

In [14]:
load_dotenv()

user_preferences = {
    "interests": ["Technology", "Science", "Art"],
    "content_types": ["Article", "Tutorial"],
    "text_format": "summary"
}

mock_content = [
    {"title": "The Future of Tech: Innovations to Watch", "category": "Technology", "type": "Article", "summary": "Explore the cutting-edge technologies shaping our future, from AI advancements to sustainable energy solutions."},
    {"title": "Beginner's Guide to Digital Art", "category": "Art", "type": "Tutorial", "summary": "Step into the world of digital art with this comprehensive tutorial for beginners. Learn about tools, techniques, and tips to start your digital art journey."},
    {"title": "Understanding Quantum Computing", "category": "Science", "type": "Article", "summary": "Dive into the complex world of quantum computing and discover how it's set to revolutionize technology with unparalleled computational power."},
    {"title": "The Renaissance Art Movement", "category": "Art", "type": "Summary", "summary": "A brief overview of the Renaissance period, highlighting its most influential artists and their timeless masterpieces."},
    {"title": "DIY Home Robotics Projects", "category": "Technology", "type": "Tutorial", "summary": "Get hands-on with technology by exploring these simple yet fascinating DIY robotics projects you can build at home."},
    {"title": "The Mysteries of Dark Matter", "category": "Science", "type": "Article", "summary": "Join us on a journey through the cosmos as we explore the enigmatic nature of dark matter and its role in the universe."},
    {"title": "Creative Writing Tips for Beginners", "category": "Art", "type": "Tutorial", "summary": "Unlock your creative potential with these essential writing tips, perfect for aspiring authors starting their journey."},
    {"title": "Innovative Eco-Friendly Technologies", "category": "Technology", "type": "Summary", "summary": "Discover how innovative technologies are paving the way for a more sustainable future, focusing on renewable energy sources and eco-friendly solutions."},
    {"title": "Exploring the Human Genome", "category": "Science", "type": "Summary", "summary": "An insightful summary of what the human genome project has taught us and how it continues to influence scientific research and medical advancements."},
    {"title": "Photography Basics: Capturing the Moment", "category": "Art", "type": "Article", "summary": "Learn the fundamentals of photography with this guide, from understanding your camera to composing the perfect shot."},
]


In [15]:
def set_user_preferences(interests, content_types):
    return {
        "interests": interests,
        "content_types": content_types
    }

# Example of setting preferences
user_preferences = set_user_preferences(["Technology", "Art"], ["Article", "Tutorial"])

print(user_preferences)


{'interests': ['Technology', 'Art'], 'content_types': ['Article', 'Tutorial']}


In [16]:


def generate_content(preferences):
    # Filter content based on user preferences
    filtered_content = [content for content in mock_content if content["category"] in preferences["interests"] and content["type"] in preferences["content_types"]]
    return filtered_content

# Example usage
personalized_content = generate_content(user_preferences)
print(personalized_content)


[{'title': 'The Future of Tech: Innovations to Watch', 'category': 'Technology', 'type': 'Article', 'summary': 'Explore the cutting-edge technologies shaping our future, from AI advancements to sustainable energy solutions.'}, {'title': "Beginner's Guide to Digital Art", 'category': 'Art', 'type': 'Tutorial', 'summary': 'Step into the world of digital art with this comprehensive tutorial for beginners. Learn about tools, techniques, and tips to start your digital art journey.'}, {'title': 'DIY Home Robotics Projects', 'category': 'Technology', 'type': 'Tutorial', 'summary': 'Get hands-on with technology by exploring these simple yet fascinating DIY robotics projects you can build at home.'}, {'title': 'Creative Writing Tips for Beginners', 'category': 'Art', 'type': 'Tutorial', 'summary': 'Unlock your creative potential with these essential writing tips, perfect for aspiring authors starting their journey.'}, {'title': 'Photography Basics: Capturing the Moment', 'category': 'Art', 'typ

In [33]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
            {"role": "system", "content": "You are an AI who writes a summary of the article."},
            {"role": "user", "content": "Say this is a test"},
        ]
        ,
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

This is a test.

# LANGCHAIN PROTOTYPE

In [25]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

In [29]:
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))


def generate_content_with_langchain(interest, content_type):
    # Craft a prompt based on user interest and desired content type
    template = """Question: {question}

    Answer: Let's think step by step."""

    prompt = PromptTemplate.from_template(template)
    question_template = """Create a {content_type} about {interest}. 
    It should be engaging and informative."""
    question = question_template.format(content_type=content_type, interest=interest)
    # Use Langchain's OpenAI LLM to generate content
    # Note: Adjust the parameters according to your requirements
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    # Extracting the generated content
    generated_content = llm_chain.run(question)

    return {
        "title": f"{interest} {content_type}",
        "content": generated_content
    }

# Example usage
interest = "Sustainable Living"
content_type = "Summary"
generated_content = generate_content_with_langchain(interest, content_type)
print(generated_content)

{'title': 'Sustainable Living Summary', 'content': " Imagine a world where the air is clean, the oceans are full of life, and the land is thriving with greenery. This is the vision of sustainable living - a lifestyle that strives to meet the present needs without compromising the ability of future generations to meet their own needs.\n\nSustainable living is a way of life that focuses on reducing our carbon footprint and preserving the Earth's resources for future generations. It involves making conscious choices in our daily activities, such as reducing energy and water consumption, using eco-friendly products, and promoting biodiversity.\n\nAt its core, sustainable living is about finding a balance between our needs and the needs of the environment. It is not about giving up modern comforts, but rather finding ways to live in harmony with nature. This can be achieved through small but impactful changes in our habits, such as using public transportation, composting, and buying locally

In [31]:
print(generated_content["content"])

 Imagine a world where the air is clean, the oceans are full of life, and the land is thriving with greenery. This is the vision of sustainable living - a lifestyle that strives to meet the present needs without compromising the ability of future generations to meet their own needs.

Sustainable living is a way of life that focuses on reducing our carbon footprint and preserving the Earth's resources for future generations. It involves making conscious choices in our daily activities, such as reducing energy and water consumption, using eco-friendly products, and promoting biodiversity.

At its core, sustainable living is about finding a balance between our needs and the needs of the environment. It is not about giving up modern comforts, but rather finding ways to live in harmony with nature. This can be achieved through small but impactful changes in our habits, such as using public transportation, composting, and buying locally-sourced products.

One of the key components of sustain

In [3]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [4]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [5]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [6]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

# TruLens

In [4]:
from dotenv import load_dotenv
load_dotenv()
# Imports main tools:
from trulens_eval import TruChain, Feedback, Tru
tru = Tru()
tru.reset_database()

# Imports from langchain to build app
import bs4
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

In [5]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

In [8]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/home/ryan/Program/myProject/personalized-content-curator/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
tru.run_dashboard()